# Autoencoder

In [ ]:
# Convolutional autoencoder for image reconstruction
# Adapted from https://keras.io/examples/vision/autoencoder/#build-the-autoencoder

## set up
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.python.keras import layers
from keras.datasets import mnist
from tensorflow.python.keras.models import Model
from PIL import Image
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Reshape

import os
from sklearn.model_selection import train_test_split


def display(array1, array2):
    """
    Displays corresponding images from each one of the supplied arrays side by side.
    """

    n = 10

    indices1 = np.random.randint(len(array1), size=n)
    indices2 = np.random.randint(len(array2), size=n)
    images1 = array1[indices1, :]
    images2 = array2[indices2, :]

    plt.figure(figsize=(20, 4))
    for i in range(n):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(images1[i].reshape(224, 224))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(images2[i].reshape(224, 224))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    plt.show()

## Model

In [ ]:
# Encoder
input_img = Input(shape=(224, 224, 1))

# Convolutional layers
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(4, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
flatten = Flatten()(x)
encoded = Dense(3136, activation='relu')(flatten)  

middle_layer = encoded

# Decoder
x = Dense(3136, activation='relu')(encoded)
reshape = Reshape((28, 28, 4))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(reshape)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='sigmoid', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)


# Autoencoder model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

In [ ]:
middle_layer.shape

## Data

In [ ]:
# Obtain all segmented png images
folder_path = '/home/xzhu/517/Toarcian AI project inc ox'

In [ ]:
# Define the dimensions of the input images
input_shape = (224, 224, 1)  # For grayscale images

# Get a list of all the image files in the directory
image_files = [filename for filename in os.listdir(folder_path) if filename.endswith('.png')]

# Calculate the number of images
num_images = len(image_files)

# Create an empty NumPy array to store images
images = np.empty((num_images, *input_shape), dtype=np.uint8)

# Load and preprocess images
for i, filename in enumerate(image_files):
    image_path = os.path.join(folder_path, filename)
    image = Image.open(image_path).convert('L')  # Convert to grayscale
    image = image.resize((input_shape[0], input_shape[1]))
    image_array = np.array(image)
    images[i] = image_array.reshape(input_shape)

# Normalize pixel values to the range [0, 1]
images = images.astype('float32') / 255.0

# Split the images into training and test sets
train_data, test_data = train_test_split(images, test_size=0.2, random_state=42)
print(train_data.shape)
print(test_data.shape)

## Training

In [ ]:
CAE = autoencoder.fit(
    x=train_data,
    y=train_data,
    epochs= 50,
    batch_size=128,
    shuffle=True,
    validation_data=(test_data, test_data),
    callbacks=[]
)

## Prediction

In [ ]:
predictions = autoencoder.predict(test_data)
display(train_data, predictions)

In [ ]:
# Get training and test loss histories
training_loss = CAE.history['loss']
val_loss = CAE.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, val_loss, 'b-')
plt.title('Autoencoder Model Loss')
plt.legend(['Training Loss', 'Validation Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
autoencoder.save('autoencoder_model2.h5')

In [ ]:
loaded_model = tf.keras.models.load_model('autoencoder_model2.h5')

In [ ]:
loaded_model.summary()

## Compression

In [ ]:
# Set the directory path
directory = '/home/xzhu/517/test'
image_size = (224, 224) 

# Get a list of all the image files in the directory
image_files = [filename for filename in os.listdir(directory) if filename.endswith('.png')]

# Create an empty list to store the images
image_list = []

# Iterate over the PNG files and read each image
for file in image_files:
    file_path = os.path.join(directory, file)
    image = cv2.imread(file_path)

    image = cv2.resize(image, image_size)

    # Convert image into binary image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Append the resized image to the list
    image_list.append(image)

# Convert the list of images to a NumPy array
image_array = np.array(image_list)

# Print the shape of the image array
print(image_array.shape)

In [ ]:
encoder = Model(inputs=loaded_model.input, outputs=loaded_model.get_layer('dense').output)
middle_layer = encoder.predict(image_array)
print(middle_layer.shape)